A célula abaixo corrige um problma com a biblioteca nltk que estávamos tendo.

In [1]:
import nltk
from nltk.tokenize import word_tokenize
# Especificar manualmente o caminho do nltk_data no ambiente virtual
nltk.data.path.append('./.venv/nltk_data')  # Ajuste esse caminho conforme necessário

# Agora tente baixar novamente os pacotes
nltk.download('punkt', download_dir='./.venv/nltk_data')
nltk.download('stopwords', download_dir='./.venv/nltk_data')
nltk.download('punkt_tab', download_dir='./.venv/nltk_data')


[nltk_data] Downloading package punkt to ./.venv/nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to ./.venv/nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt_tab to ./.venv/nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

## Limpeza dos textos
Abaixo definimos as funções usadas para limpeza do texto

In [23]:
import re
import unicodedata
from nltk.tokenize import sent_tokenize

def limpeza_texto(texto):

    # 1. Corrigir quebras de linha excessivas
    texto = texto.replace('\n', '')
    
    # 2. Corrigir múltiplos espaços
    texto = re.sub(r'[ \t]+', ' ', texto)
    
    # 3. Remover pontuação desnecessária (mas manter . , ? — e aspas)
    texto = re.sub(r'[!;:()*[\]{}<>]', '', texto)
    
    # 4. Converter para minúsculas
    texto = texto.lower()
    
    # 5. Remover acentos
    texto = unicodedata.normalize('NFKD', texto)
    texto = ''.join(c for c in texto if not unicodedata.combining(c))
    
    return texto

def limpar_e_tokenizar_texto(texto):
    # Limpa o texto primeiro
    texto_limpo = limpeza_texto(texto)
    
    sentences = sent_tokenize(texto_limpo, language='portuguese')
    # Remove sentenças vazias
    sentences = [s for s in sentences if s.strip()]
    # Tokeniza as sentenças e achata a lista de listas
    tokens = [token for s in sentences for token in word_tokenize(s, language='portuguese')]

    return tokens

def extrair_titulo_autor(nome_arquivo):
    # Remove "preprocessado_" e ".txt" ou ".json"
    nome_limpo = re.sub(r'^preprocessado_', '', nome_arquivo)
    nome_limpo = re.sub(r'\.(txt|json)$', '', nome_limpo)
    
    # Separar título e autor
    match = re.match(r'(.+?) \((.+?)\)', nome_limpo)
    if match:
        titulo, autor = match.groups()
        return titulo.strip(), autor.strip()
    else:
        return nome_limpo.strip(), None  # Se não bater, só retorna o título

### Tokenização e Armazenamento
Na célula abaixo realizamos a tokenização e armazenamento dos textos em um json com título do texto, nome do autor, e conteúdo tokenizado.

In [21]:
import os
import json

# Caminhos
caminho_txts = "data/txts"
caminho_saida_pasta = "data/textos_processados"
os.makedirs(caminho_saida_pasta, exist_ok=True)

for arquivo_nome in os.listdir(caminho_txts):
    dumped_dict = {}
    dumped_dict['titulo'] = arquivo_nome.split('(')[0].strip()
    dumped_dict['autor'] = arquivo_nome.split('(')[1].replace(')', '').replace('.txt','').strip()
    dumped_dict['tokens'] = []
    caminho_arquivo = os.path.join(caminho_txts, arquivo_nome)
    if os.path.isfile(caminho_arquivo):
        with open(caminho_arquivo, 'r', encoding='utf-8') as arquivo:
            texto_pre = arquivo.read()
            
            # Dividir em parágrafos
            frases = texto_pre.split('.')
            
            for frase in frases:
                frase = frase.strip()
                if frase:
                    tokens = limpar_e_tokenizar_texto(frase)
                    dumped_dict['tokens'].append(tokens)
            
            caminho_saida = os.path.join(caminho_saida_pasta, "preprocessado_" + arquivo_nome.replace('.txt', '.json'))
            with open(caminho_saida, 'w', encoding='utf-8') as arquivo_saida:
                json.dump(dumped_dict, arquivo_saida, ensure_ascii=False, indent=2)
            
            print(f"\nTokens estruturados salvos em: {caminho_saida}")



Tokens estruturados salvos em: data/textos_processados\preprocessado_A Cartomante (Machado de Assis).json

Tokens estruturados salvos em: data/textos_processados\preprocessado_Dom Casmurro (Machado de Assis).json

Tokens estruturados salvos em: data/textos_processados\preprocessado_O Cortiço (Aluisio de Azevedo).json

Tokens estruturados salvos em: data/textos_processados\preprocessado_O Mulato (Aluisio Azevedo).json


# A partir daqui parece teu, Baiano

In [24]:
from collections import Counter
texto = """HAMLET observa a Horácio que há mais cousas no céu e na terra do que sonha a nossa filosofia. 
Era a mesma explicação que dava a bela Rita ao moço Camilo, numa sexta-feira de novembro de 1869, quando este ria dela, por ter ido na véspera consultar uma cartomante;
a diferença é que o fazia por outras palavras."""
tokens = word_tokenize(texto, language='portuguese')
tokens = [t.lower() for t in tokens if re.match(r'[a-zA-ZáàâãéèêíìóòôõúùûçÁÀÂÃÉÈÊÍÌÓÒÔÕÚÙÛÇ]+$', t)]

def normalizar(valor, minimo, maximo, escala_min=1, escala_max=5):
    valor_normalizado = (valor - minimo) / (maximo - minimo)
    valor_normalizado = max(0, min(1, valor_normalizado)) 
    return escala_min + valor_normalizado * (escala_max - escala_min)

def diversidade_lexical(tokens):
    """
    Calcula a diversidade lexical do texto com normalização.
    
    Args:
        tokens (list): Lista de tokens do texto
        
    Returns:
        dict: Análise de diversidade lexical
    """

    # Filtrar apenas tokens alfabéticos
    tokens_alfabeticos = [t.lower() for t in tokens if re.match(r'[a-zA-ZáàâãéèêíìóòôõúùûçÁÀÂÃÉÈÊÍÌÓÒÔÕÚÙÛÇ]+$', t)]

    # Contar tokens e types
    num_tokens = len(tokens_alfabeticos)
    num_types = len(set(tokens_alfabeticos))

    if num_tokens == 0:
        return {
            'ttr': 0,
            'hapax_legomena': 0,
            'score_diversidade': 1 
        }

    # Calcular Type-Token Ratio (TTR)
    ttr = num_types / num_tokens

    # Calcular Hapax Legomena (palavras que aparecem só uma vez)
    contador = Counter(tokens_alfabeticos)
    hapax_legomena = sum(1 for palavra, freq in contador.items() if freq == 1)
    proporcao_hapax = hapax_legomena / num_tokens

    # Normalizar TTR (esperado de 0.4 a 0.8 normalmente)
    score_ttr = normalizar(ttr, minimo=0.4, maximo=0.8)

    # Normalizar Hapax (esperado de 0.3 a 0.7)
    score_hapax = normalizar(proporcao_hapax, minimo=0.3, maximo=0.7)

    # Score final (média dos dois)
    score_diversidade = (score_ttr + score_hapax) / 2

    return {
        'ttr': ttr,
        'score_ttr': score_ttr,
        'num_types': num_types,
        'num_tokens': num_tokens,
        'hapax_legomena': hapax_legomena,
        'proporcao_hapax': proporcao_hapax,
        'score_diversidade': score_diversidade,
        'score_palavras_unicas': score_hapax
    }
resultados = diversidade_lexical(tokens)
print(f"Tokens: {tokens}\n")
print(f"Resultado: {resultados}")

Tokens: ['hamlet', 'observa', 'a', 'horácio', 'que', 'há', 'mais', 'cousas', 'no', 'céu', 'e', 'na', 'terra', 'do', 'que', 'sonha', 'a', 'nossa', 'filosofia', 'era', 'a', 'mesma', 'explicação', 'que', 'dava', 'a', 'bela', 'rita', 'ao', 'moço', 'camilo', 'numa', 'de', 'novembro', 'de', 'quando', 'este', 'ria', 'dela', 'por', 'ter', 'ido', 'na', 'véspera', 'consultar', 'uma', 'cartomante', 'a', 'diferença', 'é', 'que', 'o', 'fazia', 'por', 'outras', 'palavras']

Resultado: {'ttr': 0.8214285714285714, 'score_ttr': 5, 'num_types': 46, 'num_tokens': 56, 'hapax_legomena': 41, 'proporcao_hapax': 0.7321428571428571, 'score_diversidade': 5.0, 'score_palavras_unicas': 5}


In [25]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np
texto = """HAMLET observa a Horácio que há mais cousas no céu e na terra do que sonha a nossa filosofia. 
Era a mesma explicação que dava a bela Rita ao moço Camilo, numa sexta-feira de novembro de 1869, quando este ria dela, por ter ido na véspera consultar uma cartomante;
a diferença é que o fazia por outras palavras."""
tokens = word_tokenize(texto, language='portuguese')
tokens = [t.lower() for t in tokens if re.match(r'[a-zA-ZáàâãéèêíìóòôõúùûçÁÀÂÃÉÈÊÍÌÓÒÔÕÚÙÛÇ]+$', t)]
sentences = sent_tokenize(texto, language='portuguese')
model = SentenceTransformer('neuralmind/bert-large-portuguese-cased')
def get_sentence_embeddings(sentences, model):
    """
    Obtém o embedding médio para cada sentença usando um modelo pré-treinado.

    Args:
        sentences (list): Lista de sentenças.
        model: Modelo de embedding.
        tokenizer: Tokenizer do modelo.

    Returns:
        dict: Mapeamento de sentença para vetor de embedding.
    """
    embeddings = {}
    for i, sentence in enumerate(sentences):
        if not isinstance(sentence, str) or len(sentence) < 5:
            continue
        sentence_embedding = model.encode(sentence, show_progress_bar=False)
        embeddings[sentence] = sentence_embedding
    return embeddings

def analisar_coesao_sentences(sentences, words, embeddings):
    """
    Analisa a coesão e coerência do texto de forma detalhada.

    Args:
        sentences (list): Lista de frases do texto.
        words (list): Lista de palavras do texto.
        embeddings (dict): Embeddings das sentenças (mapeando texto -> vetor).

    Returns:
        dict: Resultados de análise de coesão, conectivos e similaridade semântica.
    """
    # --- Análise de conectivos ---
    conectivos = [
        'e', 'mas', 'porém', 'contudo', 'entretanto', 'portanto', 'assim', 'logo',
        'pois', 'porque', 'já que', 'uma vez que', 'quando', 'enquanto', 'se', 'caso',
        'embora', 'apesar de', 'além disso', 'ademais', 'ou seja', 'isto é'
    ]
    conectivos_encontrados = []
    for c in conectivos:
        for sentence in sentences:
            sentence = sentence.lower()  # Normaliza para minúsculas
            if c in sentence:
                conectivos_encontrados.append(c)
                # Remove o conectivo encontrado para evitar contagem duplicada
                sentence = sentence.replace(c, '')
                break
    num_conectivos = len(conectivos_encontrados)
    num_tokens = len(words)
    proporcao_conectivos = num_conectivos / (num_tokens + 1e-6)  # Evitar divisão por zero

    # --- Análise de coesão semântica ---
    # Garante que todos os vetores de embeddings sejam válidos
    vetores = []
    for s in sentences:
        vetor = embeddings.get(s)
        if isinstance(vetor, list):
            vetores.append(np.array(vetor))
    
    if len(vetores) < 2:
        similaridade_media = 1.0  # Texto curto, assume alta coesão
    else:
        similaridades = []
        for i in range(len(vetores) - 1):
            sim = cosine_similarity(vetores[i].reshape(1, -1), vetores[i+1].reshape(1, -1))[0][0]
            similaridades.append(sim)
        similaridade_media = np.mean(similaridades)

    # --- Cálculo do Score Final ---
    # Peso 50% conectivos, 50% semântica
    score_conectivos = min(1.0, proporcao_conectivos / 0.1)  # Ideal: 10% de conectivos
    score_semantica = np.clip(similaridade_media, 0, 1)      # Garante entre 0 e 1

    coesao_score_final = (score_conectivos + score_semantica) / 2

    return {
        'coesao_score': round(coesao_score_final * 5, 2),  # Escala 0-5
        'conectivos_encontrados': conectivos_encontrados,
        'num_conectivos': num_conectivos,
        'proporcao_conectivos': round(proporcao_conectivos, 3),
        'similaridade_media': round(similaridade_media, 3),
        'num_sentencas': len(sentences)
    }
embeddings = get_sentence_embeddings(sentences, model)
resultados = analisar_coesao_sentences(sentences, tokens, embeddings)
print(f"Tokens: {tokens}\n")
print(f"Sentenças: {sentences}\n")
print(f"Resultado: {resultados}")


c:\Users\turevs\projeto-pln\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name neuralmind/bert-large-portuguese-cased. Creating a new one with mean pooling.
c:\Users\turevs\projeto-pln\.venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\turevs\.cache\huggingface\hub\models--neuralmind--bert-large-portuguese-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.

Tokens: ['hamlet', 'observa', 'a', 'horácio', 'que', 'há', 'mais', 'cousas', 'no', 'céu', 'e', 'na', 'terra', 'do', 'que', 'sonha', 'a', 'nossa', 'filosofia', 'era', 'a', 'mesma', 'explicação', 'que', 'dava', 'a', 'bela', 'rita', 'ao', 'moço', 'camilo', 'numa', 'de', 'novembro', 'de', 'quando', 'este', 'ria', 'dela', 'por', 'ter', 'ido', 'na', 'véspera', 'consultar', 'uma', 'cartomante', 'a', 'diferença', 'é', 'que', 'o', 'fazia', 'por', 'outras', 'palavras']

Sentenças: ['HAMLET observa a Horácio que há mais cousas no céu e na terra do que sonha a nossa filosofia.', 'Era a mesma explicação que dava a bela Rita ao moço Camilo, numa sexta-feira de novembro de 1869, quando este ria dela, por ter ido na véspera consultar uma cartomante;\na diferença é que o fazia por outras palavras.']

Resultado: {'coesao_score': np.float64(3.84), 'conectivos_encontrados': ['e', 'quando', 'se'], 'num_conectivos': 3, 'proporcao_conectivos': 0.054, 'similaridade_media': 1.0, 'num_sentencas': 2}
